In [1]:
import pandas as pd

# Import csv as dataframe

Dataframe Information
- df = main dataframe
- df2 = age-standarized suicide rate data
- df3 = age-standarized mental health disorders prevalence data

In [2]:
df = pd.read_csv('suicide_rates.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4392 entries, 0 to 4391
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   IndicatorCode               4392 non-null   object 
 1   Indicator                   4392 non-null   object 
 2   ValueType                   4392 non-null   object 
 3   ParentLocationCode          4392 non-null   object 
 4   ParentLocation              4392 non-null   object 
 5   Location type               4392 non-null   object 
 6   SpatialDimValueCode         4392 non-null   object 
 7   Location                    4392 non-null   object 
 8   Period type                 4392 non-null   object 
 9   Period                      4392 non-null   int64  
 10  IsLatestYear                4392 non-null   bool   
 11  Dim1 type                   4392 non-null   object 
 12  Dim1                        4392 non-null   object 
 13  Dim1ValueCode               4392 

In [3]:
df2 = pd.read_csv('suicide_rates_no_age.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549 entries, 0 to 548
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   IndicatorCode               549 non-null    object 
 1   Indicator                   549 non-null    object 
 2   ValueType                   549 non-null    object 
 3   ParentLocationCode          549 non-null    object 
 4   ParentLocation              549 non-null    object 
 5   Location type               549 non-null    object 
 6   SpatialDimValueCode         549 non-null    object 
 7   Location                    549 non-null    object 
 8   Period type                 549 non-null    object 
 9   Period                      549 non-null    int64  
 10  IsLatestYear                549 non-null    bool   
 11  Dim1 type                   549 non-null    object 
 12  Dim1                        549 non-null    object 
 13  Dim1ValueCode               549 non

In [4]:
df3 = pd.read_csv('mhd-prevalence.csv')
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   measure_id     612 non-null    int64  
 1   measure_name   612 non-null    object 
 2   location_id    612 non-null    int64  
 3   location_name  612 non-null    object 
 4   sex_id         612 non-null    int64  
 5   sex_name       612 non-null    object 
 6   age_id         612 non-null    int64  
 7   age_name       612 non-null    object 
 8   cause_id       612 non-null    int64  
 9   cause_name     612 non-null    object 
 10  metric_id      612 non-null    int64  
 11  metric_name    612 non-null    object 
 12  year           612 non-null    int64  
 13  val            612 non-null    float64
 14  upper          612 non-null    float64
 15  lower          612 non-null    float64
dtypes: float64(3), int64(7), object(6)
memory usage: 76.6+ KB


# Merging all dataframes

### Make gender indicator the same in df2 and df3

The gender indicator in df2 and df3 is as below.

| df2 | df3 |
| :-- | :-- |
| Male | Male |
| Female | Female |
| Both sexes | Both |

The difference in 'Both sexes' and 'Both' is not suitable for the merging of df2 and df3.

In [5]:
df3['sex_name'] = df3['sex_name'].apply(lambda x: x.replace('Both', 'Both sexes'))

### Add column in df2 for age-group

Unlike df1 with 'Dim2' used for the age-group column, df2 do not have an age-group column and the 'Dim2' column is empty. Therefore, we can fill 'Dim2' of df2 with 'Standarized'.

In [6]:
df2['Dim2'] = 'Standarized'
df2['Dim2']

0      Standarized
1      Standarized
2      Standarized
3      Standarized
4      Standarized
          ...     
544    Standarized
545    Standarized
546    Standarized
547    Standarized
548    Standarized
Name: Dim2, Length: 549, dtype: object

### Make sure df2 and df3 have matching country names

Since df2 and df3 are obtained from different sources, there might be some differences in the country names. We can fix this by looking for which countries in df2 do not have a match in df3.

In [7]:
# Remove some columns from df3 for simplicity
df3 = df3[['location_name', 'sex_name', 'val', 'upper', 'lower']]
df3

,location_name,sex_name,val,upper,lower
0,Belarus,Male,0.121807,0.132844,0.111691
1,Belarus,Female,0.127739,0.140663,0.116161
2,Belarus,Both sexes,0.125653,0.137332,0.115298
3,Republic of Korea,Male,0.110595,0.121588,0.100884
4,Republic of Korea,Female,0.106683,0.119142,0.096405
...,...,...,...,...,...
607,Mali,Female,0.107871,0.119061,0.097849
608,Mali,Both sexes,0.105728,0.115526,0.096618
609,United Republic of Tanzania,Male,0.119562,0.129952,0.109496
610,United Republic of Tanzania,Female,0.131263,0.144640,0.119465


In [8]:
def get_unique_diff(df2, df3):
    df2_locs = df2['Location'].unique()
    df3_locs = df3['location_name'].unique()
    print('df2 countries with no match:')
    set(df2_locs) - set(df3_locs)
    print(set(df2_locs) - set(df3_locs))
    print('\ndf3 countries with no match:')
    print(set(df3_locs) - set(df2_locs))

get_unique_diff(df2, df3)

df2 countries with no match:
{'Côte d’Ivoire', 'Türkiye', 'United Kingdom of Great Britain and Northern Ireland', 'The former Yugoslav Republic of Macedonia'}

df3 countries with no match:
{'United States Virgin Islands', 'Turkey', 'Saint Kitts and Nevis', 'Tokelau', 'American Samoa', 'Cook Islands', 'Guam', 'North Macedonia', 'Northern Mariana Islands', 'Bermuda', 'Palau', 'Taiwan (Province of China)', 'Puerto Rico', 'Tuvalu', 'Monaco', 'Niue', 'Dominica', 'Marshall Islands', "Côte d'Ivoire", 'Andorra', 'San Marino', 'Greenland', 'Palestine', 'Nauru', 'United Kingdom'}


Below are the differences in country names between df2 and df3.

| df2 | df3 |
| :-- | :-- |
| Côte d’Ivoire | Côte d'Ivoire |
| United Kingdom of Great Britain and Northern Ireland | United Kingdom |
| Türkiye | Turkey |
| The former Yugoslav Republic of Macedoniae | North Macedonia |

In [9]:
diff_loc_names = [['Côte d’Ivoire', "Côte d'Ivoire"], ['United Kingdom of Great Britain and Northern Ireland', 'United Kingdom'], ['Türkiye', 'Turkey'], ['The former Yugoslav Republic of Macedonia', 'North Macedonia']]
for i in range(len(diff_loc_names)):
    df3.loc[df3['location_name'] == diff_loc_names[i][1], 'location_name'] = diff_loc_names[i][0]
    
get_unique_diff(df2, df3)

df2 countries with no match:
set()

df3 countries with no match:
{'United States Virgin Islands', 'Saint Kitts and Nevis', 'Tokelau', 'American Samoa', 'Cook Islands', 'Guam', 'Northern Mariana Islands', 'Bermuda', 'Palau', 'Taiwan (Province of China)', 'Puerto Rico', 'Tuvalu', 'Monaco', 'Niue', 'Dominica', 'Marshall Islands', 'Andorra', 'San Marino', 'Greenland', 'Palestine', 'Nauru'}


### Merge df2 and df3

In [10]:
df_age_std = pd.merge(df2, df3,  how='left', left_on=['Location','Dim1'], right_on = ['location_name','sex_name'])
df_age_std = df_age_std.rename(columns={'val':'MhdPrevalence', 'upper':'UpperMhdPrevalence', 'lower':'LowerMhdPrevalence'})
df_age_std = df_age_std.drop(['location_name', 'sex_name'], axis=1)

### Merge df1 and df23

In [18]:
frames = [df, df_age_std]
df_clean = pd.concat(frames)
df_clean

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified,MhdPrevalence,UpperMhdPrevalence,LowerMhdPrevalence
0,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,NaN,NaN,0.0,NaN,NaN,EN,2021-07-05T16:00:00.000Z,NaN,NaN,NaN
1,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,NaN,NaN,0.0,NaN,NaN,EN,2021-07-05T16:00:00.000Z,NaN,NaN,NaN
2,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,NaN,NaN,0.0,NaN,NaN,EN,2021-07-05T16:00:00.000Z,NaN,NaN,NaN
3,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,NaN,NaN,0.0,NaN,NaN,EN,2021-07-05T16:00:00.000Z,NaN,NaN,NaN
4,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,NaN,NaN,0.0,NaN,NaN,EN,2021-07-05T16:00:00.000Z,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,MH_12,Age-standardized suicide rates (per 100 000 po...,numeric,SEAR,South-East Asia,Country,NPL,Nepal,Year,2019,...,NaN,15.22,9.77 [5.4 – 15.22],NaN,NaN,EN,2021-02-08T16:00:00.000Z,0.138272,0.150566,0.125556
545,MH_12,Age-standardized suicide rates (per 100 000 po...,numeric,AFR,Africa,Country,ZAF,South Africa,Year,2019,...,NaN,14.49,9.83 [5.75 – 14.49],NaN,NaN,EN,2021-02-08T16:00:00.000Z,0.126944,0.135548,0.118365
546,MH_12,Age-standardized suicide rates (per 100 000 po...,numeric,EUR,Europe,Country,CHE,Switzerland,Year,2019,...,NaN,12.04,9.83 [7.93 – 12.04],NaN,NaN,EN,2021-02-08T16:00:00.000Z,0.170745,0.186222,0.156933
547,MH_12,Age-standardized suicide rates (per 100 000 po...,numeric,EMR,Eastern Mediterranean,Country,BHR,Bahrain,Year,2019,...,NaN,14.38,9.86 [6.06 – 14.38],NaN,NaN,EN,2021-02-08T16:00:00.000Z,0.139643,0.151575,0.127971


In [19]:
df_clean.columns

Index(['IndicatorCode', 'Indicator', 'ValueType', 'ParentLocationCode',
       'ParentLocation', 'Location type', 'SpatialDimValueCode', 'Location',
       'Period type', 'Period', 'IsLatestYear', 'Dim1 type', 'Dim1',
       'Dim1ValueCode', 'Dim2 type', 'Dim2', 'Dim2ValueCode', 'Dim3 type',
       'Dim3', 'Dim3ValueCode', 'DataSourceDimValueCode', 'DataSource',
       'FactValueNumericPrefix', 'FactValueNumeric', 'FactValueUoM',
       'FactValueNumericLowPrefix', 'FactValueNumericLow',
       'FactValueNumericHighPrefix', 'FactValueNumericHigh', 'Value',
       'FactValueTranslationID', 'FactComments', 'Language', 'DateModified',
       'MhdPrevalence', 'UpperMhdPrevalence', 'LowerMhdPrevalence'],
      dtype='object')

### Remove empty meaningless columns

In [20]:
df_clean = df_clean[['ParentLocationCode', 
                     'ParentLocation', 
                     'SpatialDimValueCode', 
                     'Location',
                     'Period',
                     'Dim1ValueCode', 
                     'Dim1', 
                     'Dim2',
                     'FactValueNumeric',
                     'MhdPrevalence',
                     'UpperMhdPrevalence',
                     'LowerMhdPrevalence']]
df_clean

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period,Dim1ValueCode,Dim1,Dim2,FactValueNumeric,MhdPrevalence,UpperMhdPrevalence,LowerMhdPrevalence
0,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,15-24 years,0.00,NaN,NaN,NaN
1,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,25-34 years of age,0.00,NaN,NaN,NaN
2,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,35-44 years of age,0.00,NaN,NaN,NaN
3,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,45-54 years,0.00,NaN,NaN,NaN
4,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,65-74 years,0.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
544,SEAR,South-East Asia,NPL,Nepal,2019,BTSX,Both sexes,Standarized,9.77,0.138272,0.150566,0.125556
545,AFR,Africa,ZAF,South Africa,2019,FMLE,Female,Standarized,9.83,0.126944,0.135548,0.118365
546,EUR,Europe,CHE,Switzerland,2019,BTSX,Both sexes,Standarized,9.83,0.170745,0.186222,0.156933
547,EMR,Eastern Mediterranean,BHR,Bahrain,2019,MLE,Male,Standarized,9.86,0.139643,0.151575,0.127971


### Rename df_clean columns

Some of the column names in the dataframe is not clear, so renaming it will make it easier for future references.

In [22]:
df_clean = df_clean.rename({'ParentLocationCode':'who_region_id',
                            'ParentLocation':'who_region',
                            'SpatialDimValueCode':'country_id',
                            'Location':'country', 
                            'Period':'year',
                            'Dim1ValueCode':'gender_id', 
                            'Dim1':'gender', 
                            'Dim2':'age_group',
                            'FactValueNumeric': 'suicides_per_100k',
                            'MhdPrevalence':'mhd_prevalence_percent',
                            'UpperMhdPrevalence':'upper_mhd_prevalence',
                            'LowerMhdPrevalence':'lower_mhd_prevalence'}, axis=1)
df_clean

,who_region_id,who_region,country_id,country,year,gender_id,gender,age_group,suicides_per_100k,mhd_prevalence_percent,upper_mhd_prevalence,lower_mhd_prevalence
0,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,15-24 years,0.00,NaN,NaN,NaN
1,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,25-34 years of age,0.00,NaN,NaN,NaN
2,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,35-44 years of age,0.00,NaN,NaN,NaN
3,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,45-54 years,0.00,NaN,NaN,NaN
4,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,65-74 years,0.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
544,SEAR,South-East Asia,NPL,Nepal,2019,BTSX,Both sexes,Standarized,9.77,0.138272,0.150566,0.125556
545,AFR,Africa,ZAF,South Africa,2019,FMLE,Female,Standarized,9.83,0.126944,0.135548,0.118365
546,EUR,Europe,CHE,Switzerland,2019,BTSX,Both sexes,Standarized,9.83,0.170745,0.186222,0.156933
547,EMR,Eastern Mediterranean,BHR,Bahrain,2019,MLE,Male,Standarized,9.86,0.139643,0.151575,0.127971


### Modify 'AgeGroup' column

The 'AgeGroup' column do not have the same formatting for all values.

In [23]:
df_clean['age_group']

0             15-24 years
1      25-34 years of age
2      35-44 years of age
3             45-54 years
4             65-74 years
              ...        
544           Standarized
545           Standarized
546           Standarized
547           Standarized
548           Standarized
Name: age_group, Length: 4941, dtype: object

In [24]:
df_clean['age_group'] = df_clean['age_group'].apply(lambda x: x.replace(' years', ''))
df_clean['age_group'] = df_clean['age_group'].apply(lambda x: x.replace(' of age', ''))
df_clean['age_group']

0            15-24
1            25-34
2            35-44
3            45-54
4            65-74
          ...     
544    Standarized
545    Standarized
546    Standarized
547    Standarized
548    Standarized
Name: age_group, Length: 4941, dtype: object

In [25]:
def age_group_code(x):
    if x == 'Standarized':
        return 0
    elif x == '15-24':
        return 1
    elif x == '25-34':
        return 2
    elif x == '35-44':
        return 3
    elif x == '45-54':
        return 4
    elif x == '55-64':
        return 5
    elif x == '65-74':
        return 6
    elif x == '75-84':
        return 7
    elif x == '85+':
        return 8
    
df_clean['age_group_id'] = df_clean['age_group'].apply(age_group_code)
df_clean[['age_group_id', 'age_group']]

,age_group_id,age_group
0,1,15-24
1,2,25-34
2,3,35-44
3,4,45-54
4,6,65-74
...,...,...
544,0,Standarized
545,0,Standarized
546,0,Standarized
547,0,Standarized


### Final clean merged data

In [36]:
df_clean

,who_region_id,who_region,country_id,country,year,gender_id,gender,age_group,suicides_per_100k,mhd_prevalence_percent,upper_mhd_prevalence,lower_mhd_prevalence,age_group_id
0,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,15-24,0.00,NaN,NaN,NaN,1
1,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,25-34,0.00,NaN,NaN,NaN,2
2,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,35-44,0.00,NaN,NaN,NaN,3
3,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,45-54,0.00,NaN,NaN,NaN,4
4,AMR,Americas,ATG,Antigua and Barbuda,2019,BTSX,Both sexes,65-74,0.00,NaN,NaN,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,SEAR,South-East Asia,NPL,Nepal,2019,BTSX,Both sexes,Standarized,9.77,0.138272,0.150566,0.125556,0
545,AFR,Africa,ZAF,South Africa,2019,FMLE,Female,Standarized,9.83,0.126944,0.135548,0.118365,0
546,EUR,Europe,CHE,Switzerland,2019,BTSX,Both sexes,Standarized,9.83,0.170745,0.186222,0.156933,0
547,EMR,Eastern Mediterranean,BHR,Bahrain,2019,MLE,Male,Standarized,9.86,0.139643,0.151575,0.127971,0


# Separate df_clean

### Table: who_regions

In [26]:
who_regions = df_clean[['who_region_id', 'who_region']].drop_duplicates()
who_regions

,who_region_id,who_region
0,AMR,Americas
40,WPR,Western Pacific
77,EUR,Europe
86,AFR,Africa
108,SEAR,South-East Asia
118,EMR,Eastern Mediterranean


In [27]:
who_regions.to_csv('who_regions.csv', index=False)

### Table: countries

In [28]:
countries = df_clean[['country_id', 'country', 'who_region_id']].drop_duplicates()
countries

,country_id,country,who_region_id
0,ATG,Antigua and Barbuda,AMR
22,BRB,Barbados,AMR
40,BRN,Brunei Darussalam,WPR
52,CRI,Costa Rica,AMR
54,GRD,Grenada,AMR
...,...,...,...
693,EST,Estonia,EUR
709,UKR,Ukraine,EUR
845,MNG,Mongolia,WPR
925,BDI,Burundi,AFR


In [29]:
countries.to_csv('countries.csv', index=False)

### Table: genders

In [30]:
genders = df_clean[['gender_id', 'gender']].drop_duplicates()
genders

,gender_id,gender
0,BTSX,Both sexes
7,FMLE,Female
14,MLE,Male


In [31]:
genders.to_csv('genders.csv', index=False)

### Table: age_groups

In [32]:
age_groups = df_clean[['age_group_id', 'age_group']].drop_duplicates()
age_groups = age_groups.sort_values('age_group_id')
age_groups

,age_group_id,age_group
0,0,Standarized
0,1,15-24
1,2,25-34
2,3,35-44
3,4,45-54
18,5,55-64
4,6,65-74
5,7,75-84
6,8,85+


In [33]:
age_groups.to_csv('age_groups.csv', index=False)

### Table: suicide_records

In [34]:
suicide_records = df_clean[['country_id', 'gender_id', 'age_group_id', 'suicides_per_100k', 'mhd_prevalence_percent']]
suicide_records

,country_id,gender_id,age_group_id,suicides_per_100k,mhd_prevalence_percent
0,ATG,BTSX,1,0.00,NaN
1,ATG,BTSX,2,0.00,NaN
2,ATG,BTSX,3,0.00,NaN
3,ATG,BTSX,4,0.00,NaN
4,ATG,BTSX,6,0.00,NaN
...,...,...,...,...,...
544,NPL,BTSX,0,9.77,0.138272
545,ZAF,FMLE,0,9.83,0.126944
546,CHE,BTSX,0,9.83,0.170745
547,BHR,MLE,0,9.86,0.139643


In [35]:
suicide_records.to_csv('suicide_records.csv', index=False)